# Regression with FACET

***

**Robust and impactful Data Science with FACET**

FACET enables us to perform several critical steps in best practice Data Science work flow easily, efficiently and reproducibly:

1. Create a robust pipeline for learner selection using LearnerRanker and cross-validation.

2. Enhance our model inspection to understand drivers of predictions using local explanations of features via SHAP values by applying a novel methodology that decomposes SHAP values into measures of synergy, redundancy, and independence between each pair of features.

3. Quickly apply historical simulation to gain key insights into feature values that minimize or maximize the predicted outcome.

***

**Context**

The data are from simulated experiments of a naval vessel (Frigate) Gas Turbine (GT) propulsion plant. The simulator considers the performance decay over time of the GT components such as GT compressor and turbines. One observation in this dataset represents the current decay states of the compressor and the gas turbine along with several sensor readings of the vessel at that point in time. The decay state is a performance metric [0, 1] where 1 means delivering 100% of nominal performance. We want to determine the machine settings which maximize the gas turbine decay state coefficient.

Utilizing FACET, we will: 

1. Predict the decay state of the gas turbine as accurately as possible
2. Understand which parameters drive the decay state of the turbine
3. Analyse how these drivers interact with each other and the target

While we can solve questions 1 and parts of question 2 with commonly used machine learning packages, FACET will enable us to make better inferences about the way some of the features share or complement information and help us to figure out the optimal settings of the vessel to minimize the equipment degradation at a variety of ship speeds.

***

**Tutorial outline**

1. [Required imports](#Required-imports)
2. [Preprocessing and initial feature selection](#Preprocessing-and-initial-feature-selection)
3. [Selecting a learner using FACET ranker](#Selecting-a-learner-using-FACET-ranker)
4. [Using the FACET inspector for model inspection](#Using-the-FACET-inspector-for-model-inspection)
5. [FACET univariate simulator: the impact of waist to height ratio](#FACET-univariate-simulator:-the-impact-of-waist-to-height-ratio)
6. [Appendix](#Appendix)

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx


def _set_paths() -> None:
    # set the correct path when launched from within PyCharm

    module_paths = ["pytools", "facet", "sklearndf"]

    import sys
    import os

    if "cwd" not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
        os.chdir(cwd)
    print(f"working dir is '{os.getcwd()}'")
    for module_path in module_paths:
        if module_path not in sys.path:
            sys.path.insert(0, os.path.abspath(f"{cwd}/{os.pardir}/{module_path}/src"))
        print(f"added `{sys.path[0]}` to python paths")


def _ignore_warnings():
    # ignore irrelevant warnings that would affect the output of this tutorial notebook

    # ignore a useless LGBM warning
    import warnings

    warnings.filterwarnings("ignore", category=UserWarning, message=r".*Xcode_8\.3\.3")


_set_paths()
_ignore_warnings()

del _set_paths, _ignore_warnings

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx


def _configure_matplotlib():
    # set global options for matplotlib

    import matplotlib

    matplotlib.rcParams["figure.figsize"] = (16.0, 8.0)
    matplotlib.rcParams["figure.dpi"] = 72


_configure_matplotlib()

del _configure_matplotlib

# Required imports

In order to run this notebook, we will import not only the FACET package, but also other packages useful to solve this task. Overall, we can break down the imports into three categories: 

1. Common packages (pandas, matplotlib, etc.)
2. Required FACET classes (inspection, selection, validation, simulation, etc.)
3. Other BCG Gamma packages which simplify pipelining (sklearndf, see on [GitHub](https://github.com/BCG-Gamma/sklearndf/)) and support visualization (pytools, see on [GitHub](https://github.com/BCG-Gamma/pytools)) when using FACET

**Common package imports**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from sklearn.model_selection import RepeatedKFold

**Gamma FACET imports**

In [ ]:
from facet.data import Sample
from facet.crossfit import LearnerCrossfit
from facet.inspection import LearnerInspector
from facet.selection import LearnerRanker, LearnerGrid
from facet.validation import BootstrapCV
from facet.simulation.partition import ContinuousRangePartitioner
from facet.simulation import UnivariateUpliftSimulator
from facet.simulation.viz import SimulationDrawer

**sklearndf imports**

Instead of using the "regular" scikit-learn package, we are going to use sklearndf (see on [GitHub](https://github.com/BCG-Gamma/sklearndf/)). sklearndf is an open source library designed to address a common issue with scikit-learn: the outputs of transformers are numpy arrays, even when the input is a data frame. However, to inspect a model it is essential to keep track of the feature names. sklearndf retains all the functionality available through scikit-learn plus the feature traceability and usability associated with Pandas data frames. Additionally, the names of all your favourite scikit-learn functions are the same except for `DF` on the end. For example, the standard scikit-learn import:

`from sklearn.pipeline import Pipeline`

becomes:

`from sklearndf.pipeline import PipelineDF`  

In [ ]:
from sklearndf.pipeline import PipelineDF, RegressorPipelineDF
from sklearndf.regression import RandomForestRegressorDF
from sklearndf.regression.extra import LGBMRegressorDF
from sklearndf.transformation.extra import BorutaDF
from sklearndf.transformation import SimpleImputerDF

**pytools imports**

pytools (see on [GitHub](https://github.com/BCG-Gamma/pytools)) is an open source library containing general machine learning and visualization utilities, some of which are useful for visualising the advanced model inspection capabilities of FACET.

In [ ]:
from pytools.viz.dendrogram import DendrogramDrawer, DendrogramReportStyle
from pytools.viz.distribution import ECDFDrawer
from pytools.viz.matrix import MatrixDrawer

# Preprocessing and initial feature selection

First, we need to load our turbine decay data and create a simple preprocessing pipeline. For those interested initial EDA can be found in the Appendix [Exploratory Data Analysis](#Exploratory-Data-Analysis)

In [ ]:
# load the prepared data frame
decay_df = pd.read_csv(
    "sphinx/source/tutorial/gas_turbine_data.txt", delim_whitespace=True
)

# assign human-readable labels
decay_df.columns = [
    "Lever Position",
    "Ship Speed",
    "Turbine Shaft Torque (kN m)",
    "Turbine Rate of Revolutions (rpm)",
    "Generator Rate of Revolutions (rpm)",
    "Starboard Propeller Torque (kN)",
    "Port Propeller Torque (kN)",
    "HP Turbine exit temp (C)",
    "GT Compressor inlet air temp (C)",
    "GT Compressor outlet air temp (C)",
    "Turbine exit pressure (bar)",
    "Compressor inlet air pressure (bar)",
    "Compressor outlet air pressure (bar)",
    "Turbine exhaust gas pressure (bar)",
    "Turbine injection control",
    "Fuel flow",
    "GT Compressor decay state coeff",
    "GT Turbine decay state coeff",
]

# need to drop the other target
decay_df.drop("GT Compressor decay state coeff", inplace=True, axis=1)

# to ensure quick running we will use a random sample of 2500 observations
decay_df = decay_df.sample(n=2500, random_state=42)

# have a look
decay_df.head()

For easier data management we will create a sample object using FACET's `Sample` class, which allows us to: 

- Quickly access the target vs. features
- Pass our data into sklearndf pipelines
- Pass information to other FACET functions

In [ ]:
decay_obs = Sample(observations=decay_df, target_name="GT Turbine decay state coeff")

Next, we should create a minimum preprocessing pipeline. However, based on our EDA ([Exploratory Data Analysis](#Exploratory-Data-Analysis-(EDA)) ) we have no missing values or a need to manage categorical variables etc. However, while it is not needed we will create a simple imputation preprocessing pipeline using sklearndf's `SimpleImputerDF` to demonstrate how such a pipeline is created and included in subsequent analytic steps.

In [ ]:
feature_preprocessing = PipelineDF(steps=[("impute", SimpleImputerDF())])

Next, we perform some initial feature selection using Boruta, a recent approach shown to have quite good performance. The Boruta algorithm removes features that are no more predictive than random noise. If you are interested further, please see this  [article](https://www.jstatsoft.org/article/view/v036i11).

The `BorutaDF` transformer in sklearndf package provides easy access to this powerful method. The approach relies on a tree-based learner, usually a random forest. For settings, a `max_depth` of between 3 and 7 is typically recommended, and here we rely on the default setting of 5. However, as this depends on the number of features and the complexity of interactions one could also explore the sensitivity of feature selection to this parameter. The number of trees is automatically managed by the Boruta feature selector argument `n_estimators="auto"`.

We also use parallelization for the random forest using `n_jobs` to accelerate the Boruta iterations.

In [ ]:
# create the Boruta object
boruta_feat_select = BorutaDF(
    estimator=RandomForestRegressorDF(max_depth=5, random_state=42, n_jobs=-3),
    n_estimators="auto",
    max_iter=50,
    random_state=42,
    verbose=False,
)

# combine Boruta with the preprocessing pipeline
selection_pipeline = PipelineDF(
    steps=[
        ("preprocessing", feature_preprocessing),
        ("feature selection", boruta_feat_select),
    ]
)

Since [sklearndf](https://github.com/BCG-Gamma/sklearndf) closely follows the `scikit-learn` syntax, we can fit this pipeline on the `sample.features` and `sample.target` properties.

In [ ]:
# fit pipeline and print selected features
selection_pipeline.fit(X=decay_obs.features, y=decay_obs.target)
print(f"Selected features: {selection_pipeline.feature_names_original_.unique()}")

Boruta selected 10 features which we will now keep in our FACET sample object. Note that this feature selection process could be included in a general preprocessing pipeline, however due to the computation involved, we have utilized Boruta here as an initial one-off processing step to narrow down the features for our classifier development.

In [ ]:
# update FACET sample object to only those features Boruta identified as useful
decay_obs_initial_features = decay_obs.keep(
    feature_names=selection_pipeline.feature_names_original_.unique()
)

# Selecting a learner using FACET ranker

FACET implements several additional useful wrappers which further simplify comparing and tuning a larger number of models and configurations: 

- `LearnerGrid`: allows you to pass a learner pipeline (i.e., regressor + any preprocessing) and a set of hyperparameters
- `LearnerRanker`: multiple LearnerGrids can be passed into this class as a list - this allows tuning hyperparameters both across different types of learners in a single step and ranks the resulting models accordingly

The following learners and hyperparameter ranges will be assessed using 10 repeated 5-fold cross-validation:


1. **Random forest**: with hyperparameters
    - min_samples_leaf: [8, 16]
    - n_estimators: [20, 50, 100]  
  
  
2. **Light gradient boosting**: with hyperparameters
    - min_data_in_leaf: [8, 16]
    - subsample: [0.8, 1]
    - boosting_type: ["gbdt"]
    
Note if you want to see a list of hyperparameters you can use `regressor_name().get_params().keys()` where `regressor_name` could be for example `RandomForestRegressorDF` and if you want to see the default values, just use `regressor_name().get_params()`.

Finally, for this exercise we will use the default performance metric for scoring and ranking our regressors, which is explained variance. Note that ranking uses the average performance minus two times the standard deviation, so that we consider both the average performance and variability when selecting a regressor.

First, we specify the classifiers we want to train using `RegressorPipelineDF` from sklearndf. Note here we could also include any feature preprocessing steps by using the `preprocessing` argument.

In [ ]:
rforest_regressor = RegressorPipelineDF(
    regressor=RandomForestRegressorDF(random_state=42),
)

lgbm_regressor = RegressorPipelineDF(
    regressor=LGBMRegressorDF(random_state=42),
)

Then we create a list of learner grids where each learner grid is created using `LearnerGrid` and allows us to associate a `RegressorPipelineDF` with a specified set of hyperparameter via the `learner_parameters` argument. Note this structure allows us to easily include additional regressors and hyperparameters.

In [ ]:
regressor_grid = [
    LearnerGrid(
        pipeline=rforest_regressor,
        learner_parameters={"min_samples_leaf": [8, 16], "n_estimators": [20, 50, 100]},
    ),
    LearnerGrid(
        pipeline=lgbm_regressor,
        learner_parameters={
            "min_data_in_leaf": [8, 16],
            "subsample": [0.8, 1],
            "boosting_type": ["gbdt"],
        },
    ),
]

We now fit the grid defined above using the `LeanerRanker`, which will run a gridsearch (or random search if defined) using 10 repeated 5-fold cross-validation on our selected set of features from Boruta.

In [ ]:
regressor_ranker = LearnerRanker(
    grids=regressor_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=42),
    n_jobs=-3,
    verbose=False,
    scoring="explained_variance",
).fit(sample=decay_obs_initial_features)

We can see how each model scored using the `summary_report()` method of the `LearnerRanker`.

In [ ]:
# let's look at performance for the top 5 ranked regressors
regressor_ranker.summary_report()

We can see based on our learner ranker, we have selected a LGBM algorithm that achieved a mean explained variance of 0.96 with a SD of 0.01.

# Using the FACET inspector for model inspection

The [SHAP approach](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) has become the standard method for model inspection. SHAP values are used to explain the additive contribution of each feature to the prediction for a given observation. SHAP values are computed for every feature and observation.

The FACET `LearnerInspector` computes SHAP values for each crossfit (i.e., a CV fold or bootstrap resample) using the best model identified by the `LearnerRanker`. The FACET `LearnerInspector` then provides advanced model inspection through new SHAP-based summary metrics for understanding feature redundancy and synergy. Redundancy and synergy are calculated using the new algorithm FACET implements for using SHAP values to understand model predictions.

The definitions are as follows:

- **Redundancy** represents how much information is shared between two features contributions to the model predictions. For example, temperature and pressure in a pressure cooker are redundant features for predicting cooking time since pressure will rise relative to the temperature, and vice versa. Therefore, knowing just one of either temperature or pressure will likely enable the same predictive accuracy. Redundancy is expressed as a percentage ranging from 0% (full uniqueness) to 100% (full redundancy).


- **Synergy** represents how much the combined information of two features contributes to the model predictions. For example, given features X and Y as coordinates on a chess board, the colour of a square can only be predicted when considering X and Y in combination. Synergy is expressed as a percentage ranging from 0% (full autonomy) to 100% (full synergy).

Both cases can apply at the same time, i.e. a pair of features can use some information synergistically while using other information redundantly.

To analyse redundancy for all possible feature parings, the approach is:

1.	Calculate the feature redundancy matrix using SHAP value decomposition - this gives us pairwise redundancy between features, in the range of 0.0 (fully unique contributions) and 1.0 (fully redundant contributions)
2.	Transform the feature redundancy matrix into a feature distance matrix, where distance is expressed as (1.0 - redundancy)
3.	Perform hierarchical, single-linkage clustering on the distance matrix, thus identifying groups of low-distance, redundant features which activate “in tandem” to predict the outcome

The same approach can be used to analyse synergy.

The inspector can calculate all of this with a single method call, but also offers methods to access the intermediate results of each step. A lightweight visualization framework is available to render the results in different styles.

SHAP values from the `LearnerInspector` can also be used with the SHAP package plotting functions for sample and observation level SHAP visualizations, such as SHAP distribution plots, dependency plots, force plots and waterfall plots.

In [ ]:
# run inspector
regression_inspector = LearnerInspector()
regression_inspector.fit(
    n_jobs=-3, verbose=False, crossfit=regressor_ranker.best_model_crossfit_
)

## Synergy and redundancy

Synergy and redundancy are part of the key extensions FACET makes to using SHAP values to understand model predictions.

In [ ]:
# synergy heatmaps
synergy_matrix = regression_inspector.feature_synergy_matrix()
MatrixDrawer(style="matplot%").draw(synergy_matrix, title="Feature synergies")

*Synergy represents the degree to which one feature combines with another to generate a prediction.* We can see that the feature "Compressor outlet air pressure (bar)" is synergistic with other features. This means that the outlet air pressure **in combination** with rate of revolutions and generator rate of revolutions carries information for the model.

In [ ]:
# redundancy heatmap
redundancy_matrix = regression_inspector.feature_redundancy_matrix()
MatrixDrawer(style="matplot%").draw(redundancy_matrix, title="Feature redundancies")

*Redundancy represents the shared information between two features.* We can see that this uncovers high redundancy between `Turbine injection control` and `Fuel flow`, as well as some redundancy for other feature pairs such as (`Turbine exhaust gas pressure` and `Turbine Rate of Revolutions`) and (`Turbine injection control` and `Compressor outlet air pressure`). Another way to look at redundancy is using a dendrogram.

In [ ]:
# redundancy dendrogram
redundancy_linkage = regression_inspector.feature_redundancy_linkage()
DendrogramDrawer().draw(title="Redundancy linkage", data=redundancy_linkage)

For convenience when working in a non-notebook environment, all of the `Drawer`s provided by the [pytools](https://github.com/BCG-Gamma/pytools) package also support a `style='text'` flag.  

In [ ]:
DendrogramDrawer(style="text").draw(title="Redundancy linkage", data=redundancy_linkage)

To obtain the hierarchical clustering, we calculate a linkage tree and plug it into a dendrogram drawer. This makes it easy to visually single out features that are both important and mutually redundant.

## What can we learn from the above? 


**Synergy**

We can see in the synergy matrix that the feature "Compressor outlet air pressure (bar)" is synergistic with other features. This means that the outlet air pressure **in combination** with rate of revolutions and generator rate of revolutions carries information for the model. 

**Redundancy**

The redundancy matrix and dendrogram reveals a "cluster" of two variables which are highly redundant - Turbine injection control and Fuel flow. That is, they provide the same information to the target and are likely dependent on each other. Looking at the process, the fuel flow is a consequence from the turbine injection control, so we can remove this redundant feature. 

In turn, we remove the redundant features from the sample and create a revised sample on which we re-train the pipeline.

In [ ]:
redundant_features = ["Fuel flow", "Turbine exhaust gas pressure (bar)"]
decay_obs_no_redundant = decay_obs_initial_features.drop(redundant_features)

In [ ]:
# run the training pipeline again
regressor_ranker = LearnerRanker(
    grids=regressor_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=42),
    n_jobs=-3,
    verbose=False,
).fit(sample=decay_obs_no_redundant)

We can also check the best ranked model after removing redundant features.

In [ ]:
regressor_ranker.best_model_

# FACET univariate simulator: the impact of compressor outlet air temp

Another advantage of FACET is the ability to quickly instigate and run univariate simulations. Simulation enables us to gain insight into what value(s) of this compressor outlet air temp might minimize performance decay. Further, because FACET can use bootstrap cross validation, we can create a crossfit from our previous `LearnerRanker` best model to perform the simulation and to also quantify the uncertainty by using bootstrap confidence intervals.

From the synergy matrix, we can see that the Compressor outlet temperature has the highest synergy with most other features. Therefore, we want to see how the target behaves if we simulate this feature such that each state had a constant outlet compressor temperature.

The absolute SHAP values also confirm that the three features most synergistic with `GT Compressor outlet air temp (C)` are also the most important features according to the model:

In [ ]:
abs_shap_values = (
    regression_inspector.shap_values()
    .abs()
    .sum(axis=0)
    .reset_index()
    .rename({0: "Sum of abs SHAP values"}, axis=1)
)
abs_shap_values.sort_values(by="Sum of abs SHAP values", ascending=False)

As the basis for the simulation, we divide the feature into relevant partitions: 

- We use FACET's `ContinuousRangePartitioner` to split the range of observed values of the outlet air temperature into intervals of equal size. Each partition is represented by the central value of that partition. 
- For each partition, the simulator creates an artificial copy of the original sample assuming the variable to be simulated has the same value across all observations - which is the value representing the partition. Using the best `LearnerCrossfit` acquired from the ranker, the simulator now re-predicts all targets using the models trained for all folds and determines the average uplift of the target variable resulting from this.
- The FACET `SimulationDrawer` allows us to visualise the result; both in a matplotlib and a plain-text style

Finally, because FACET can use bootstrap cross validation, we can create a crossfit from our previous `LearnerRanker` best model to perform the simulation so we can quantify the uncertainty by using bootstrap confidence intervals.

In [ ]:
boot_crossfit = LearnerCrossfit(
    pipeline=regressor_ranker.best_model_.native_estimator,
    cv=BootstrapCV(n_splits=250, random_state=42),
    n_jobs=-3,
    verbose=False,
).fit(sample=decay_obs_no_redundant)

In [ ]:
# set-up and run a simulation
SIM_FEATURE = "GT Compressor outlet air temp (C)"
GTC_air_temp_simulator = UnivariateUpliftSimulator(crossfit=boot_crossfit, n_jobs=3)
GTC_air_temp_partitions = ContinuousRangePartitioner()
GTC_air_temp_simulation = GTC_air_temp_simulator.simulate_feature(
    name=SIM_FEATURE, partitioner=GTC_air_temp_partitions
)

In [ ]:
# visualize the results
SimulationDrawer().draw(data=GTC_air_temp_simulation, title=SIM_FEATURE)

In [ ]:
# can also get a print out of simulation results
SimulationDrawer("text").draw(data=GTC_air_temp_simulation, title=SIM_FEATURE)

We can see from this that the turbine decay state coefficient is maximized when the outlet air temperature of the gas turbine is as small as possible. Note that this is only looking at the partitions of the historically observed range, as extrapolating these predictions into unobserved regions would risk creating infeasible scenarios. 

# Summary

With the capabilities offered by FACET we were able to:

1. Identify a learner using with good performance for predicting performance decay.
2. Utilize advanced the SHAP value capabilities (synergy and redundancy) to identify additional features that could be removed (i.e., fuel flow and turbine exhaust gas pressure) and whether any features had strong synergistic effects - which in this case was primarily between turbine exhaust gas pressure and turbine rate of revolutions.
3. Simulate the effect of changes in GT compressor outlet air temp on performance decay.

# Appendix

## Data source

**Dataset**

The dataset used in this example is available on [Kaggle](https://www.kaggle.com/elikplim/maintenance-of-naval-propulsion-plants-data-set) and contains data from experiments carried out by means of a numerical simulator of a naval vessel (Frigate) characterized by a **Gas Turbine propulsion plant**. In this release of the simulator it is also possible to consider the performance decay over time of the GT components such as GT compressor and turbines.

Each possible degradation state of the plant can be characterized by three parameters: 

- Ship speed (linear function of the lever position)
- Compressor degradation coefficient kMc
- Turbine degradation coefficient

The degradation coefficients typically vary between [1; 0.95] for compressor and [1; 0.975] for the gas turbine. 

**Features**

One observation in this dataset represents the current decay states of the compressor and the gas turbine along with several sensor readings of the shipping vessels at that point in time. 

Our target, the Gas Turbine decay  state is being modelled as a performance decay state metric which is measured as 1 to 0, 1 meaning delivering 100% of the nominal performance. Therefore, we want to determine the machine settings which **maximize the gas turbine decay state coefficient**. 


**Learning Problem**

For this learning problem, we have three key objectives: 

1. Predict the decay state of the gas turbine as accurately as possible
2. Understand which parameters drive the decay state of the turbine
3. Analyse how these drivers interact with each other and the target


While we can solve questions 1 and parts of question 2 with commonly used machine learning packages, `facet` will enable us to make better inferences about the way some of the features share or complement information and help us to figure out the optimal settings of the vessel to minimize the equipment degradation at a variety of ship speeds.

Reference for the dataset used in this example is: A. Coraddu, L. Oneto, A. Ghio, S. Savio, D. Anguita, M. Figari, Machine Learning Approaches for Improving Condition? Based Maintenance of Naval Propulsion Plants, Journal of Engineering for the Maritime Environment, 2014, DOI: 10.1177/1475090214540874, (In Press)

## Exploratory Data Analysis (EDA)

In [ ]:
decay_df = pd.read_csv(
    "sphinx/source/tutorial/gas_turbine_data.txt", delim_whitespace=True
)

decay_df.columns = [
    "Lever Position",
    "Ship Speed",
    "Turbine Shaft Torque (kN m)",
    "Turbine Rate of Revolutions (rpm)",
    "Generator Rate of Revolutions (rpm)",
    "Starboard Propeller Torque (kN)",
    "Port Propeller Torque (kN)",
    "HP Turbine exit temp (C)",
    "GT Compressor inlet air temp (C)",
    "GT Compressor outlet air temp (C)",
    "Turbine exit pressure (bar)",
    "Compressor inlet air pressure (bar)",
    "Compressor outlet air pressure (bar)",
    "Turbine exhaust gas pressure (bar)",
    "Turbine injection control",
    "Fuel flow",
    "GT Compressor decay state coeff",
    "GT Turbine decay state coeff",
]

TARGET = "GT Turbine decay state coeff"

In [ ]:
# need to drop the other target
decay_df.drop("GT Compressor decay state coeff", inplace=True, axis=1)

In [ ]:
# describe the data
decay_df.describe().T

We can also quickly check for missing values, although we can see that there appears to be none.

In [ ]:
# count missing by feature
decay_df.isna().sum(axis=0)

We can use pytool's `ECDFDrawer()` to draw the cumulative distribution of the target. This shows us that the target is uniformly distributed in increments of 0.001 increments.

In [ ]:
# look at the target distribution
decay_obs = Sample(observations=decay_df, target_name=TARGET)
ECDFDrawer().draw(decay_obs.target)

In [ ]:
# look at feature distributions and correlations
sns.pairplot(decay_obs.features)

**Quick EDA summary:**

- We have no missing values in our data
- We are dealing with a linearly distributed target
- The features exhibit a mixture of linear and non-linear relationships amongst each other. This gives us reason to test several non-parametric models and compare their performance